# Test attack on (Signed) PEP for weakly-self dual codes, using squares of hulls

We generate a pair of codes having length $n$, dimension $k$ and hull dimension $h$, defined over a finite field with $q$ elements.

We first generate the generator matrix $\mathbf G\in\mathbb F_q^{k\times n}$ of a random code with such parameters, then sample a random change of basis $\mathbf S\in GL_k(\mathbb F_q)$ and a random permutation matrix $\mathbf P\in\mathbb F_q^{n\times n}$ and set

$\mathbf G' = \mathbf S\cdot \mathbf G\cdot \mathbf P$

The script optionally tests also the case of Signed PEP. To do this, set $\mathtt{signed}\_\mathtt{PEP} = \mathtt{True}$. In such a case, we sample also a random diagonal matrix $\mathbf D\in\mathbb F_q^{n\times n}$ with coefficients $\pm 1$ and set 

$\mathbf G' = \mathbf S\cdot \mathbf G\cdot \mathbf P\cdot \mathbf D$

In [1]:
#simulate the attack
import time

load('code_utils.sage')
load('ABL_sample_weakly_self_dual.sage')
load("GIP_solver.sage")

# Generate first code

You may do this by importing one of the codes from the folder (if you want to simulate the attack on the ABL scheme), or you can generate a new code. Notice that generation of large weakly-self dual codes takes some time...

In [1]:
import csv

n = 7313
k = 450
q = 8191
h = 27
signed_PEP = True #set to True if you want to test the attack on Signed PEP

id_code = 0

folder_name = "codes_"+str(n)+"_"+str(k)+"_"+str(q)+"_"+str(h)

file_name = "G_"+str(n)+"_"+str(k)+"_"+str(q)+"_"+str(h)+"_"+str(id_code)

Fq = GF(q)
G = matrix(Fq,k,n)
with open(folder_name+"/"+file_name, newline='\n') as csvfile:
    G_reader = csv.reader(csvfile, delimiter=',')
    i = 0
    for row in G_reader:
        for j in range(n):
            G[i,j] = int(row[j])
        i += 1

In [2]:
n = 100
k = 50
q = 127
h = 10
signed_PEP = True #set to True if you want to test the attack on Signed PEP
verbose = False

G = faster_sample_weakly_self_dual_code(q, n, k, h, verbose)

In [3]:
print("Rank(G) = ",rank(G))
print("Hull dim. = ",k-rank(G*G.transpose()))

Rank(G) =  50
Hull dim. =  10


# Sample instance of PEP

In [5]:
Fq = GF(q) #finite field

#sample change of basis
rank_S = 0
while rank_S < k:
    S = random_matrix(Fq, k, k)
    rank_S = rank(S)

#sample random permutation
P_as_list = Permutations(n).random_element()
P = P_as_list.to_matrix().change_ring(Fq)

#sample random diagonal (if signed_PEP = False, this is the identity matrix)
D = identity_matrix(Fq,n)

if signed_PEP:
    for i in range(n):
        val = Set([Fq(1), Fq(-1)]).random_element()
        D[i,i] = val
        
#compute G'
G_prime = S*G*P*D

print("PEP instance generated")

PEP instance generated


In [7]:
####compute hulls
M1 = hull(Fq, G)
M2 = hull(Fq, G_prime)

##square hulls
U1 = square_code(M1) #basis of square of hull for first code
U2 = square_code(M2) #basis of square of hull for second code 

#Compute hulls and print their dimension
#hull_U1 = hull(Fq, U1)
#hull_U2 = hull(Fq, U2)
print("Hull dimension of U1 = ",rank(U1)-rank(U1*U1.transpose()))
print("Hull dimension of U2 = ",rank(U2)-rank(U2*U2.transpose()))

#compute graphs
start = time.time()
A1 = graph_from_generator_matrix(U1)
A2 = graph_from_generator_matrix(U2)
end = time.time()
time_gauss = end - start
print("--> Time for computing graphs = ",time_gauss," s")

#solve GIP
start = time.time()
sol = simple_solve_GIP(Fq, A1, A2)
end = time.time()
time_GIP = end - start
print("--> Time for solving GIP = ",time_GIP," s")

print("Reduction to GIP concluded, total time = ",time_gauss + time_GIP," s")

#Check validity of found permutation
my_P = matrix(Fq, n, n)
for i in range(n):
#    print(sol[i])
    j1 = sol[i][0]
    j2 = sol[i][1]
    my_P[j1, j2] = 1

if my_P == P:
    print("We found the correct permutation!")
else:
    print("Found permutation is wrong, sorry :(")
    
##find D
perm_G = G*my_P

#set up matrix
num_G = 100
num_H = 100
A = matrix(Fq,k*(n-k),n)
num = 0

H_prime = codes.LinearCode(G_prime).parity_check_matrix()

pos_G = Combinations(k,num_G).random_element()
for i in pos_G:
    pos_H = Combinations(n-k,num_H).random_element()
    for j in pos_H:
        for ell in range(n):
            A[num,ell] = perm_G[i,ell]*H_prime[j,ell]
        num += 1

#find kernel of A
solution_space = A.right_kernel()
B = solution_space.basis()
print("Dimension of solution space = ",len(B))

b = B[0]

b1 = b[0]^-1 * b
b2 = -b1

#see if b1 is ok
ok1 = 1
i = 0
while (ok1 == 1)&(i<n):
    if b1[i]==D[i,i]:
        i += 1
    else:
        ok1 = 0

#see if b2 is ok
ok2 = 1
i = 0
while (ok2 == 1)&(i<n):
    if b2[i]==D[i,i]:
        i += 1
    else:
        ok2 = 0

#check validity of solution
if (ok1 == 1)|(ok2 == 1):
    print("We retrieved also the scalar coefficients!")
else:
    print("Something went wrong!")
    
time_fin = time.time()

print("Tot. time = ",time_fin - time_in)

Hull dimension of U1 =  0
Hull dimension of U2 =  0
--> Time for computing graphs =  0.0027420520782470703  s
--> Time for solving GIP =  0.018518686294555664  s
Reduction to GIP concluded, total time =  0.021260738372802734  s
we found the correct permutation!


ValueError: empty range for randrange() (0, 0, 0)

In [7]:
print("--> Is the found permutation the same as the initial solution?",P == my_P)

--> Is the found permutation the same as the initial solution? True


# Try finding D without using all equations

In [6]:
#set up matrix
num_G = 400
num_H = 400
A = matrix(Fq,num_G*num_H,n)
num = 0

perm_G = G*my_P
H_prime = codes.LinearCode(G_prime).parity_check_matrix()

pos_G = Combinations(k,num_G).random_element()
for i in pos_G:
    pos_H = Combinations(n-k,num_H).random_element()
    for j in pos_H:
        for ell in range(n):
            A[num,ell] = perm_G[i,ell]*H_prime[j,ell]
        num += 1

#find kernel of A
solution_space = A.right_kernel()
B = solution_space.basis()
print("Dimension of solution space = ",len(B))

KeyboardInterrupt: 

### (Don't look below here!) Compute $\mathbf G'$

In [8]:
Fq = GF(q) #finite field

#sample change of basis
rank_S = 0
while rank_S < k:
    S = random_matrix(Fq, k, k)
    rank_S = rank(S)

#sample random permutation
P_as_list = Permutations(n).random_element()
P = P_as_list.to_matrix().change_ring(Fq)

#sample random diagonal (if signed_PEP = False, this is the identity matrix)
D = identity_matrix(Fq,n)

if signed_PEP:
    for i in range(n):
        val = Set([Fq(1), Fq(-1)]).random_element()
        D[i,i] = val
        
#compute G'
G_prime = S*G*P*D

### Start the attack

We now simulate our attack.

##### Compute the hulls
We consider only the hulls of the two codes, with bases $\mathbf M_1$ and $\mathbf M_2$

In [9]:
load('code_utils.sage')
#load('ABL_sample_weakly_self_dual.sage')

M1 = hull(Fq, G)
M2 = hull(Fq, G_prime)

#Sanity check: verify that the solution permutation maps the hulls
#res = codes.LinearCode(M1*P*D) == codes.LinearCode(M2)
#print("Does P*D send M1 to M2?", res)

In [10]:
matrix(M1.basis())

AttributeError: 'sage.matrix.matrix_modn_dense_double.Matrix_modn_dense_double' object has no attribute 'basis'

##### Compute squares of the hulls

We also check the hull dimension of the squares. If hulls are trivial, the attack is rather fast.
If hulls are non trivial, then the attack may take more time.

In [11]:
load('code_utils.sage')
U1 = square_code(M1) #basis of square of hull for first code
U2 = square_code(M2) #basis of square of hull for second code 

#Compute hulls and print their dimension
#hull_U1 = hull(Fq, U1)
#hull_U2 = hull(Fq, U2)
print("Hull dimension of U1 = ",rank(U1)-rank(U1*U1.transpose()))
print("Hull dimension of U2 = ",rank(U2)-rank(U2*U2.transpose()))

Hull dimension of U1 =  0
Hull dimension of U2 =  0


##### Solve via reduction to GIP

We call our simple GIP solver and compare the found solution with the true solution.

We first compute only the permutation.

In [12]:
load("GIP_solver.sage")
import time

#compute graphs
start = time.time()
A1 = graph_from_generator_matrix(U1)
A2 = graph_from_generator_matrix(U2)
end = time.time()
time_gauss = end - start
print("--> Time for computing graphs = ",time_gauss," s")

#solve GIP
start = time.time()
sol = simple_solve_GIP(Fq, A1, A2)
end = time.time()
time_GIP = end - start
print("--> Time for solving GIP = ",time_GIP," s")

print("Reduction to GIP concluded, total time = ",time_gauss + time_GIP," s")


--> Time for computing graphs =  6.925938129425049  s
--> Time for solving GIP =  228.54900193214417  s
Reduction to GIP concluded, total time =  235.4749400615692  s


Check validity of solution: we see if the found permutation matches with the true permutation

In [ ]:
my_P = matrix(Fq, n, n)
for i in range(n):
#    print(sol[i])
    j1 = sol[i][0]
    j2 = sol[i][1]
    my_P[j1, j2] = 1

#see if permutation maps the hulls
#print("--> Does the found permutation solves PEP on the hulls?", codes.LinearCode(M1*my_P*D) == codes.LinearCode(M2))

#see if permutation maps the initial codes
#print("--> Does the found permutation solves PEP on the codes?",codes.LinearCode(G*my_P*D) == codes.LinearCode(G_prime))

#see if permutations are the same
print("--> Is the found permutation the same as the initial solution?",P == my_P)

###### Only for Signed PEP

Now, recover also the scalar coefficients.
We have found the permutation $\mathbf P$. 
We take a parity-check matrix $\mathbf H'$ from $\mathbf G'$ and then set the system

$\mathbf G\cdot \begin{pmatrix}x_1 & 0 & \cdots & 0\\
0 & x_2 & \cdots & 0\\
\vdots & \vdots & \ddots & \vdots \\
0 & 0 & \cdots & x_n\end{pmatrix}\cdot \mathbf H'^\top = \mathbf 0$

To solve the system, we set up the matrix 

$\mathbf A = \left\{\widetilde{\mathbf g}_i\star \mathbf h'_j\right\}_{\begin{smallmatrix}1\leq i\leq k\\ 1 \leq j \leq n-k\end{smallmatrix}}$

with $\widetilde{\mathbf g}_i = i$-th row of $\mathbf G\cdot \mathbf P$, $\mathbf h'_j = j$-th row of $\mathbf H$, and then find its right kernel. 

In [ ]:
perm_G = G*my_P

#set up matrix

num_G = 100
num_H = 70
A = matrix(Fq,k*max_u,n)
num = 0

H_prime = codes.LinearCode(G_prime).parity_check_matrix()

pos_G = Combinations(k,num_G).random_element()
for i in pos_G:
    pos_H = Combinations(n-k,num_H).random_element()
    for j in pos_H:
        for ell in range(n):
            A[num,ell] = perm_G[i,ell]*H_prime[j,ell]
        num += 1

#find kernel of A
solution_space = A.right_kernel()
B = solution_space.basis()
print("Dimension of solution space = ",len(B))

When the kernel has dimension 1, it is enough to check if its generator, i.e., a vector $\mathbf b\in\mathbb F_q^n$ has a scalar multiple which is composed only of values $\pm 1$.

To do this, we "normalize" the vector (we multiply it by the inverse of the first entry) by computing $\mathbf b' = b_1^{-1}\cdot \mathbf b$ and check if either $\mathbf g$ or $-\mathbf g'$ correspond to the diagonal of $\mathbf d$.

In [ ]:
b = B[0]

b1 = b[0]^-1 * b
b2 = -b1

#see if b1 is ok
ok1 = 1
i = 0
while (ok1 == 1)&(i<n):
    if b1[i]==D[i,i]:
        i += 1
    else:
        ok1 = 0

#see if b2 is ok
ok2 = 1
i = 0
while (ok2 == 1)&(i<n):
    if b2[i]==D[i,i]:
        i += 1
    else:
        ok2 = 0

#check validity of solution
if (ok1 == 1)|(ok2 == 1):
    print("We retrieved also the scalar coefficients!")
else:
    print("Something went wrong!")

In [ ]:
n*(n-k)*k*13/8.

If the kernel has dimension $>1$, we search for a valid elementin the kernel.
(Not implemented yet!!)